In [1]:
! curl "http://zero.borm.org:8000"

Not found
This is a 4store SPARQL server 
Check /status/ for more info


import vcf

# REPLACE pjotr.me with your name!!

In [3]:
print("@prefix : <http://sem.tamara/rdf/ns#> .\n")
import vcf 

vcf_reader = vcf.Reader(open('/home/user/gatk_exome.vcf', 'r'))
count=0
for record in vcf_reader:
    name = "chr"+record.CHROM+"_"+str(record.POS)+"_"+str(record.ALT[0])
    print(":"+name)
    print("  :query_id \""+name+"\" ;")
    print("  :chr \""+record.CHROM+"\" ;")
    print("  :pos \""+str(record.POS)+"\" ;")
    print("  :alt \""+str(record.ALT[0])+"\" .")
    count +=1
    if count > 5:
        break


@prefix : <http://sem.tamara/rdf/ns#> .

:chr1_22848972_C
  :query_id "chr1_22848972_C" ;
  :chr "1" ;
  :pos "22848972" ;
  :alt "C" .
:chr1_33133968_C
  :query_id "chr1_33133968_C" ;
  :chr "1" ;
  :pos "33133968" ;
  :alt "C" .
:chr1_60466814_C
  :query_id "chr1_60466814_C" ;
  :chr "1" ;
  :pos "60466814" ;
  :alt "C" .
:chr1_60466840_T
  :query_id "chr1_60466840_T" ;
  :chr "1" ;
  :pos "60466840" ;
  :alt "T" .
:chr1_62516683_C
  :query_id "chr1_62516683_C" ;
  :chr "1" ;
  :pos "62516683" ;
  :alt "C" .
:chr1_66058513_G
  :query_id "chr1_66058513_G" ;
  :chr "1" ;
  :pos "66058513" ;
  :alt "G" .


### Now write the same to file with samples and a few extra triples

In [6]:

import vcf

buf = """
@prefix    :   <http://sem.tamara/rdf/ns#> .
@prefix rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

"""

vcf_reader = vcf.Reader(open('/home/user/gatk_exome.vcf', 'r'))
for record in vcf_reader:
    name = "chr"+record.CHROM+"_"+str(record.POS)+"_"+str(record.ALT[0])
    buf += ":"+name+"\n"
    buf += "  :variant \""+name+"\" ;\n"
    buf += "  rdf:type :snp ;\n"
    buf += "  :chr \""+record.CHROM+"\" ;\n"
    buf += "  :pos \""+str(record.POS)+"\" ;\n"
    buf += "  :alt \""+str(record.ALT[0])+"\" .\n"
    for sample in record.samples:
        buf += ":"+name+"_"+str(sample.sample)+"\n"
        if sample["DP"]:
            buf += "  :dp   "+str(sample["DP"])+" ;\n"
        buf += "  :vcf :"+name+" .\n"

# Finally we connect a few SNPs

buf += """
:chr1_66058513_G  :controls  :chr1_114354942_G .
:chr1_114354942_G :controls  :chr1_144873962_C .
:chr1_144873962_C :controls  :chr1_66058513_G .
:chr1_144873962_C :controls  :chrX_134713855_A .
"""



# Write to file

print(buf)
f = open("test_sem.rdf","w") # note this file is shared!
f.write(buf)



### Validate the file - we can do this in python

In [7]:
# pip3 install --user rdflib
# python3 rdfvalidate.py

import rdflib
g = rdflib.Graph()

result = g.parse('test_sem.rdf', format='n3')
print(len(g))
for stmt in g:
    print(stmt)

# You should see something here...

### Here we upload the data to the database

In [8]:
# Equivalent of curl -T $rdf -H 'Content-Type: application/x-turtle' $uri
import requests

myname="sem.tamara" # change to your username
rdf="test_sem.rdf" # shared file?!
uri="http://zero.borm.org:8000/data/http://biobeat.org/"+myname+"/"+rdf
headers = {'content-type': 'application/x-turtle'}

f = open(rdf)
buf = f.read()
r = requests.put(uri, data=buf, headers=headers)
print(uri)
print(r) # should give a 201


http://zero.borm.org:8000/data/http://biobeat.org/sem.tamara/test_sem.rdf
<Response [201]>


### Let's check what is in 4store


In [9]:
! lynx --dump zero.borm.org:8000/status/size/

                       SPARQL httpd server status - size

KB pythoncourse

   Segment # quads (s) quads (sr) models resources
   0         12584     +0         17     8164
   1         12411     +0         17     8149
     Total   24995     +0         17     16313


### And do a first SPARQL query


In [10]:

end_point = "http://zero.borm.org:8000/sparql/"

query = """
PREFIX : <http://sem.tamara/rdf/sem.tamara/ns#>
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?s WHERE {
    ?s rdf:type ?o .
} LIMIT 10
"""

r = requests.post(end_point, data={ "query": query, "output": "text" })
# print r.url

print(r.text)


?s
<http://zyzhang.me/rdf/ns#chr19_3434413_G>
<http://sem.tamara/rdf/ns#chr19_6413578_T>
<http://jurrian.me/rdf/ns#chr22_43936228_T>
<http://sem.tamara/rdf/ns#chr1_33133968_C>
<http://ianthe.me/rdf/ns#chr19_22952111_G>
<http://ethel.me/rdf/ns#chr1_66058513_G>
<http://tobi.me/rdf/ns#chr16_82033612_C>
<http://tobi.me/rdf/ns#chr16_82033612_C>
<http://astrid.me/rdf/ns#chr3_122437321_C>
<http://emma.me/rdf/ns#chr1_60466814_C>



In [11]:
#count number of entries with chromosome 1
end_point = "http://zero.borm.org:8000/sparql/"

query = """
PREFIX sem: <http://sem.tamara/rdf/ns#>
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT 
(COUNT(?chr) AS ?numchr1)
WHERE {
    
    ?chr sem:chr "1".
     
} 
"""

r = requests.post(end_point, data={ "query": query, "output": "text" })
# print r.url

print(r.text)


?numchr1
18



In [12]:
#cross reference query
end_point = "http://zero.borm.org:8000/sparql/"

query = """
PREFIX sem: <http://sem.tamara/rdf/ns#>
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT 
?controlled
?controller
WHERE {
    
    
    ?controller sem:controls ?controlled.
     
} 
"""

r = requests.post(end_point, data={ "query": query, "output": "text" })
# print r.url

print(r.text)


?controlled	?controller
<http://sem.tamara/rdf/ns#chr1_144873962_C>	<http://sem.tamara/rdf/ns#chr1_114354942_G>
<http://sem.tamara/rdf/ns#chr1_114354942_G>	<http://sem.tamara/rdf/ns#chr1_66058513_G>
<http://sem.tamara/rdf/ns#chr1_66058513_G>	<http://sem.tamara/rdf/ns#chr1_144873962_C>
<http://sem.tamara/rdf/ns#chrX_134713855_A>	<http://sem.tamara/rdf/ns#chr1_144873962_C>



In [13]:
!/ext/anaconda3/bin/pip install --user urllib2

  Could not find a version that satisfies the requirement urllib2 (from versions: )


No matching distribution found for urllib2


In [26]:

import csv


url = 'https://raw.githubusercontent.com/stamara6/AdvBioInf/master/bpe_matched_lowres_final.csv'

r = requests.get(url)
text = r.iter_lines()
reader = csv.reader(text, delimiter=',')


In [64]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/stamara6/AdvBioInf/master/bpe_matched_lowres_final.csv')
data['matchname'] = data['matchname'].str.replace('>','')

In [65]:
data

In [70]:
rdf_buf = """
@prefix    :   <http://sem/rdf/ns#> .
@prefix rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

"""

for index, row in data.iterrows():
    #print(row["Average_Mass"])
    name = row["matchname"]
    rdf_buf += ":" + name + "\n"
    rdf_buf += "  :numpeb \"" + str(row["numpeb"])+"\" ;\n"
    rdf_buf += "  :ox \"" + str(row["ox"]) +"\" ;\n"
    rdf_buf += "  :subunit \"" + row["subunit"] + "\" .\n"


print(rdf_buf)
f = open("bpe.rdf","w")
f.write(rdf_buf)






@prefix    :   <http://sem/rdf/ns#> .
@prefix rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

:alphaminus0residues2peb
  :numpeb "2" ;
  :ox "0" ;
  :subunit "alpha" .
:betaminus0residues3pebMethyl
  :numpeb "3" ;
  :ox "0" ;
  :subunit "beta" .
:betaminus0residues3pebMethyl1Ox
  :numpeb "3" ;
  :ox "1" ;
  :subunit "beta" .
:betaminus0residues3pebMethyl
  :numpeb "3" ;
  :ox "0" ;
  :subunit "beta" .
:betaminus0residues3pebMethyl2Ox
  :numpeb "3" ;
  :ox "2" ;
  :subunit "beta" .
:alphaminus0residues1peb
  :numpeb "1" ;
  :ox "0" ;
  :subunit "alpha" .
:gamma4minus49residues5peb
  :numpeb "5" ;
  :ox "0" ;
  :subunit "gamma4" .
:betaminus0residues3pebMethyl2Ox
  :numpeb "3" ;
  :ox "2" ;
  :subunit "beta" .
:gamma4minus50residues5peb
  :numpeb "5" ;
  :ox "0" ;
  :subunit "gamma4" .
:gamma4minus48residues5peb
  :numpeb "5" ;
  :ox "0" ;
  :subunit "gamma4" .
:alphaminus5residues3peb
  :numpeb "3" ;
  :ox "0" ;
  :subunit "alpha" .
:betaminus1residues1peb
  :numpeb "1" ;
  :ox

6087

In [71]:
import requests

myname="sem"
rdf="bpe.rdf"
uri="http://zero.borm.org:8000/data/http://biobeat.org/"+myname+"/"+rdf
headers = {'content-type': 'application/x-turtle'}

f = open(rdf)
rdf_buf = f.read()
r = requests.put(uri, data=rdf_buf, headers=headers)
print(uri)
print(r) # should give a 201


http://zero.borm.org:8000/data/http://biobeat.org/sem/bpe.rdf
<Response [201]>


In [98]:
#select only gamma subunits and output number of chromophores and numer of oxidations encountered
end_point = "http://zero.borm.org:8000/sparql/"

query = """
PREFIX sem: <http://sem/rdf/ns#>
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT
?subunit
?numpeb
?ox
WHERE {
        
    ?name sem:subunit  ?subunit
    FILTER regex(?subunit, "^gamma")
    
    ?name sem:numpeb ?numpeb;
     sem:ox ?ox.
    
} 

"""

r = requests.post(end_point, data={ "query": query, "output": "text" })
# print r.url

print(r.text)


?subunit	?numpeb	?ox
"gamma4"	"5"	"3"
"gamma2"	"4"	"2"
"gamma4"	"5"	"2"
"gamma1"	"5"	"2"
"gamma4"	"5"	"1"
"gamma2"	"4"	"2"
"gamma1"	"5"	"1"
"gamma4"	"5"	"1"
"gamma2"	"5"	"0"
"gamma2"	"3"	"0"
"gamma4"	"5"	"1"
"gamma2"	"5"	"2"
"gamma1"	"5"	"3"
"gamma3"	"3"	"2"
"gamma3"	"3"	"3"
"gamma1"	"5"	"0"
"gamma2"	"5"	"1"
"gamma4"	"5"	"1"
"gamma1"	"2"	"2"
"gamma4"	"5"	"2"
"gamma2"	"6"	"0"
"gamma3"	"3"	"1"
"gamma4"	"5"	"2"
"gamma3"	"2"	"2"
"gamma4"	"4"	"0"
"gamma4"	"5"	"1"
"gamma2"	"4"	"3"
"gamma2"	"5"	"2"
"gamma4"	"5"	"0"
"gamma3"	"3"	"2"
"gamma1"	"5"	"1"
"gamma2"	"4"	"1"
"gamma4"	"5"	"0"
"gamma4"	"5"	"0"
"gamma3"	"3"	"1"
"gamma4"	"5"	"2"
"gamma2"	"3"	"1"
"gamma4"	"5"	"2"
"gamma1"	"5"	"0"
"gamma1"	"5"	"2"
"gamma2"	"4"	"0"
"gamma2"	"5"	"1"
"gamma4"	"5"	"1"
"gamma1"	"5"	"1"
"gamma2"	"5"	"0"
"gamma4"	"5"	"3"
"gamma3"	"3"	"2"
"gamma3"	"3"	"1"
"gamma1"	"2"	"0"
"gamma2"	"5"	"2"
"gamma4"	"5"	"2"
"gamma4"	"5"	"0"
"gamma3"	"3"	"2"

